# Content Based Recommendation Systems

Recommendation systems are the systems that users predict future prefecences by using their past experiences and prefences. RS uses to a lot domains such as social media and e-commerce. Percent of 80 being watched movies and series on Netflix and percent of 60 being watched videos on Youtube has been explored by using thanks to recommendation systems. Amazon is in debt to recommendations systems percent of 35 of its gain. 

Recommendation systems are divided into three:

* Collaborative Filtering
* Content Based RS
* Hybrid Models

In this notebook we are going to discuss content based RS.

## Content Based Recommendation Systems

* CB are the systems that create a recommendation system by capturing similarity relationship between items according to comments, description or attributes of items which users interact. 
* It is used to models such as TF_IDF and Word2Vec in order to capture similarity.
* It is very powerful that a item adding newly is recommend. 
* CB Filtering Approachs always provides apparent recommendation in order to use content descriprions. For example, if users have never used items having specific key words, it has no chance of recommendation. 
* The reason for this, designed model is always the special the user at hand. So experience of similar users is not benefited. This tends to reduce the variety of non-ideal recommended items. 
* Other disavantage in CB, although it is so powerful for present item recommendation, it is not successful in new user recommendation. In general they create hybrid model by combining content based filtering and collaborative filtering. 

## What is TF-IDF?

We use to capture similarity between content (in text data such as atribute, type etc.) TF-IDF or Word2Vec model. We need to understand this topic in order to set up right recommendation system. So what is tf-idf?

TF-IDF is a weight factor which a word display important into document and had been calculated with statistics method. TF-IDF method use to a lot domains (sentiment analysis, RS, stop words filterin etc.) This method is divided into two. Fistly we will analyze Term Frequency(TF).

### Term Frequency

It express odds a word that is found document. For example "Jandino: Whatever it Takes" is a document. Odds of "Jandino" is 1/4 (number of Jandino / number of words in document)

### Inverse Document Frequency

It express logaritmic value of the ratio number of document and number of the word that contains document.

**TF-IDF method used as multipy TF value and IDF value. (TF * IDF)**

I applied this method in my model. And I found between similarity with cosine distance. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For Text
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Capture similarity 
from sklearn.metrics.pairwise import linear_kernel

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


Let's get to know our dataset.

In [2]:
data = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


> I wanted to make a suggestion depends on the cast members, description and "listed_in" which contains the shows type but there is another column named "type" it is a bit confusing I know. > 

Drop nan values on these columns to make a proper matrix which contains linear_kernel values of selected strings.

In [3]:
data.dropna(subset=['cast','title','description','listed_in'],inplace=True,axis=0)
data = data.reset_index(drop=True)

Let get to know feature that we will use:

* type: media type (movie, tv show etc.)
* title: media name
* listed_in: place where it listed.
* description: information about media

In order to apply content based recommendation system, fistly we will clean text data and combine them.

In [4]:
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(',',' ',re.sub(' ','',t)) for t in data['cast']]
data['description'] = [re.sub(r'[^\w\s]', '', t) for t in data['description']]
data['title'] = [re.sub(r'[^\w\s]', '', t) for t in data['title']]

In [5]:
data["combined"] = data['listed_in'] + '  ' + data['cast'] + ' ' + data['title'] + ' ' + data['description']
data.drop(['listed_in','cast','description'],axis=1,inplace=True)
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,combined
0,s2,TV Show,Blood Water,NaN,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows TV Dramas TV Mysteries ...
1,s3,TV Show,Ganglands,Julien Leclercq,NaN,"September 24, 2021",2021,TV-MA,1 Season,Crime TV Shows International TV Shows TV Actio...
2,s5,TV Show,Kota Factory,NaN,India,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows Romantic TV Shows TV Co...
3,s6,TV Show,Midnight Mass,Mike Flanagan,NaN,"September 24, 2021",2021,TV-MA,1 Season,TV Dramas TV Horror TV Mysteries KateSiegel Z...
4,s7,Movie,My Little Pony A New Generation,"Robert Cullen, José Luis Ucha",NaN,"September 24, 2021",2021,PG,91 min,Children Family Movies VanessaHudgens Kimiko...


In [6]:
# Content Similarity
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])
cosine_similarities = linear_kernel(matrix,matrix)
movie_title = data['title']
indices = pd.Series(data.index, index=data['title'])

Now Let's set up our recommendation model.

In [7]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movie_title.iloc[movie_indices]

Let's try our model whether works right.

In [8]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return  movie_title.iloc[movie_indices]

Here is our recommendations.

In [9]:
content_recommender('The Crown')

709                                         Downton Abbey
543                                             The Queen
5283                          Witches A Century of Murder
6665                                           London Spy
6848                                       My Hotter Half
4797                                                Reign
7457      The Blue Planet A Natural History of the Oceans
7689                          The Real Football Factories
4528                                           Collateral
4625                                             Lovesick
7030                 Planet Earth The Complete Collection
5546                                         Age Gap Love
4566                          The Frankenstein Chronicles
3859                                 Single Ladies Senior
7458                                       The Bomb Squad
6879                         Natures Great Events Diaries
3888                                              Flowers
5542          

If you want to save it into csv:

In [10]:
title = 'The Crown'
suggestions = content_recommender(title)

suggestions_df = pd.DataFrame(data=suggestions)
suggestions_df.to_csv('suggestions_based_on_%s.csv'%title,index=False,header=False)

In this above. Our model brought the recommendations. We wanted to bring acording to Transformes series movie. Firstly it brought to Transformers movies.

# Conclusion

* In this notebook. I worked to explain content based recommendation system.
* Content based RS models are powerful in new item recommendation. 
* In general it is used TF-IDF and Word2Vec model while content based RS designs.
* TF-IDF method is the method which explains words frequency in document.
* It can be created a recommendation model by using cosine distance with the aid of TF-IDF weights.

If you benefit this notebook. Please upvote for my motivastion. I will be waiting your mind in comments. Your comments are so important to improve my skills. Thanks to your votes and comments.